In [2]:
import os

os.environ["COHERE_API_KEY"] = 'E2efronXbGqXSR0sclyInSjXptFOjdIjsbCeXFwP'

from langchain_cohere import ChatCohere
from langchain_ollama import ChatOllama

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate

In [4]:
from pydantic import BaseModel, Field

In [5]:
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
# screening abstracts

In [11]:

class OutputSchema(BaseModel):
    thoughts: str = Field(description= "thoughts of the model")
    decision: int = Field(description="1 if the abstract was selected, 0 otherwise")
    reason: str = Field(description = "generate a consise one sentence long reason for the decision")


llm = ChatCohere(model="command-r", temperature=0.0)
# llm = ChatOllama(model="deepseek-r1:32b", temperature=0.0)
struct_llm = llm.with_structured_output(OutputSchema)

In [ ]:
df = pd.read_csv("articles.csv")[["key", "title", "abstract"]]

display(df.head())
print(len(df))

In [ ]:
messages = [
    ('system',  "You are a helpful AI agent that assists in accurately screening the article using its abstract. " 
                "Based solely on the abstract provided, determine whether the article discusses an APPLICATION OF AI METHODS IN CARBON ION THERAPY? "
                "Your decision should be '0' for NO or '1' for YES. Then, generate a concise, one-sentence reason for your decision."
    ),
    ('human', "abstract:\n\n title: {title}, \n content: {abstract}")
  ]

prompt_template = ChatPromptTemplate.from_messages(messages)
chain = prompt_template | struct_llm

decision_df = {"key":[], "title":[], "abstract":[], "decision":[], "reason":[], "thoughts":[]}

rows = [row.to_dict() for i,row in df.iterrows()]

for row in tqdm(rows):
  # key, title, selection = row.to_dict()["key"], row.to_dict()["title"], row.to_dict()["selection"]

  inputs = {key:row[key] for key in ["title", "abstract"]} 

  output = chain.invoke(inputs)

  for key,val in {**row, **dict(output)}.items():
      decision_df[key].append(val)
  
decision_df = pd.DataFrame(decision_df)
decision_df.to_csv("ai_screening_outputs_final.csv", index=False)


In [ ]:
# generating summaries

In [ ]:

class OutputSchema(BaseModel):
    thoughts: str = Field(description= "thoughts of the model")
    study_objective: str = Field(description="The objective of the article")
    ai_methods: str = Field(description="The AI method(s) used in the article in the context of carbon ion therapy")
    application_domain: str = Field(description="carbon ion therapy application domain")
    dataset_characteristics: str = Field(description="characteristics of the dataset, including training, validation, test, etc")
    key_findings: str = Field(description="key findings mentioned in the article")
    pros_and_cons: str = Field(description="strengths and limitations of the study")
    future_direction: str = Field(description="future directions (if stated)")
# llm = ChatCohere(model="command-r", temperature=0.0)
# llm = ChatCohere(model='command-r', temperature=0.0)
llm = ChatOllama(model="deepseek-r1:32b", temperature=0.0)
struct_llm = llm.with_structured_output(OutputSchema)



In [ ]:
file_path = r"1-s2.0-S1120179724002163-main.pdf"

In [ ]:
# from langchain_community.document_loaders import PyPDFLoader


# loader = PyPDFLoader(file_path)
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

# manuscript = "\n".join([page.page_content for page in pages])

In [ ]:
import pymupdf4llm

manuscript = pymupdf4llm.to_markdown(file_path)

ref_index = manuscript.lower().find("references")

# Keep everything up to and including "References"
if ref_index != -1:
    manuscript = manuscript[:ref_index + len("references")]

# Now md_text contains only up to the References section
print(manuscript)

In [ ]:
messages = [
    ('system',  "You are a helpful AI agent that assists in accurately summarizing the article in a standardized format "
                "to be used for the literature review titled 'APPLICATIONS OF AI IN CARBON ION THERAPY'. \n"
                "Summarize the article under the following standard headers:\n"
                "'1. Study objective', '2. AI methods used', '3. carbon therapy application domain', '4. Dataset characteristics', '5. Key findings', "
                "'6. Strengths and limitations', '7. Future directions (if stated)' "
    ),
    ('human', "article: \n {article}")
  ]

prompt_template = ChatPromptTemplate.from_messages(messages)
chain = prompt_template | struct_llm

In [ ]:
output = chain.invoke({"article":manuscript})

In [ ]:
dict(output)

# https://www.reddit.com/r/LangChain/comments/1e7cntq/whats_the_best_python_library_for_extracting_text/